In [1]:
import zarr
import numpy as np
from utils import write_tools
from utils.write_tools import flatten_3d_list

In [2]:
array_cube_side = 2048
raw_ncar_folder_path = '/home/idies/workspace/turb/data02_02/ncar-high-rate-fixed-dt'
dest_folder_name = "sabl2048b"  # B is the high-rate data
write_type = "prod"  # or "back" for backup

In [3]:
def verify_512_cube_data(original_512, zarr_512_path):
    zarr_512 = zarr.open_group(zarr_512_path, mode='r')
    for var in original_512.data_vars:
        np.testing.assert_array_equal(original_512[var].values, zarr_512[var][:])

In [4]:
timestep_nr = 1

queue = []
cubes, _ = write_tools.prepare_data(raw_ncar_folder_path + "/jhd." + str(timestep_nr).zfill(3) + ".nc")
cubes = flatten_3d_list(cubes)
dests = write_tools.get_512_chunk_destinations(dest_folder_name, write_type, timestep_nr, array_cube_side)

for i in range(len(dests)):
    queue.append((cubes[i], dests[i]))


Started preparing NetCDF data for verification. This will take ~20min
Done preparing data. Starting to verify...


In [5]:
i=0

written_512 = zarr.open_group(dests[i], mode='r')
original_512 = cubes[i]

In [6]:
from dask.array.utils import assert_eq

In [7]:
type(written_512['energy'])

zarr.core.Array

In [8]:
type(original_512['energy'])

xarray.core.dataarray.DataArray

In [9]:
import dask.array as da

dask_array_from_zarr = da.from_zarr(written_512['energy'])

In [10]:
type(dask_array_from_zarr)

dask.array.core.Array

In [11]:
dask_array_from_xarray = original_512['energy'].data

In [12]:
type(dask_array_from_xarray)

dask.array.core.Array

In [13]:
assert_eq(dask_array_from_xarray, dask_array_from_zarr)

True

In [ ]:
from dask.distributed import Client

# Create a local Dask cluster with a specified number of workers
client = Client(n_workers=4)  # For example, 4 workers


In [21]:
from dask import delayed, compute

def test_data_comparison():
    for original_512, zarr_512_path in zip(cubes, dests):
        print(zarr_512_path)
        verify_512_cube_data(original_512, zarr_512_path)

        
def verify_512_cube_data(original_512, zarr_512_path):
    zarr_512 = zarr.open_group(zarr_512_path, mode='r')
    
    a = assert_eq(original_512[var].data, da.from_zarr(zarr_512[var]))
    
    # Compute all the delayed objects in parallel
    computed_results = compute(*results)
    
    for var, is_equal in computed_results:
        print(f"Variable: {var}, Equal: {is_equal}")

    return computed_results

In [24]:
test_data_comparison()

/home/idies/workspace/turb/data01_01/zarr/sabl2048b_01_prod/sabl2048b01_001.zarr
Variable: temperature, Equal: True
Variable: pressure, Equal: True
Variable: energy, Equal: True
Variable: velocity, Equal: True
/home/idies/workspace/turb/data02_01/zarr/sabl2048b_02_prod/sabl2048b02_001.zarr


Process Dask Worker process (from Nanny):
2023-09-11 16:36:26,947 - distributed.nanny - ERROR - Worker process died unexpectedly
Process Dask Worker process (from Nanny):
2023-09-11 16:36:26,948 - distributed.nanny - ERROR - Worker process died unexpectedly
2023-09-11 16:36:26,948 - distributed.nanny - ERROR - Worker process died unexpectedly
Traceback (most recent call last):
  File "/home/idies/mambaforge/envs/py39/lib/python3.9/multiprocessing/process.py", line 315, in _bootstrap
    self.run()
  File "/home/idies/mambaforge/envs/py39/lib/python3.9/multiprocessing/process.py", line 108, in run
    self._target(*self._args, **self._kwargs)
  File "/home/idies/mambaforge/envs/py39/lib/python3.9/site-packages/distributed/process.py", line 188, in _run
    target(*args, **kwargs)
  File "/home/idies/mambaforge/envs/py39/lib/python3.9/site-packages/distributed/nanny.py", line 981, in _run
    asyncio.run(run())
Traceback (most recent call last):
  File "/home/idies/mambaforge/envs/py39/l

KeyboardInterrupt: 

2023-09-11 16:36:31,048 - distributed.nanny - WARNING - Restarting worker
2023-09-11 16:36:31,061 - distributed.nanny - WARNING - Restarting worker
2023-09-11 16:36:31,070 - distributed.nanny - WARNING - Restarting worker
